```
███████  █████  ███████ ████████ ████████ ██████   █████  ███    ██ ███████ ███████ ███████ ██████  
██      ██   ██ ██         ██       ██    ██   ██ ██   ██ ████   ██ ██      ██      ██      ██   ██ 
█████   ███████ ███████    ██       ██    ██████  ███████ ██ ██  ██ ███████ █████   █████   ██████  
██      ██   ██      ██    ██       ██    ██   ██ ██   ██ ██  ██ ██      ██ ██      ██      ██   ██ 
██      ██   ██ ███████    ██       ██    ██   ██ ██   ██ ██   ████ ███████ ██      ███████ ██   ██ 

# Demo 1 - Source Oracle - Target PGSQL17

In [1]:
dotnet tool install --global Microsoft.dotnet-interactive --version 1.0.632301

Tool 'microsoft.dotnet-interactive' is already installed.


In [2]:
#r "nuget:Microsoft.DotNet.Interactive.PostgreSQL , *-*"

Installed Packages Microsoft.DotNet.Interactive.PostgreSQL, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\romai\.nuget\packages\microsoft.dotnet.interactive.postgresql\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query PostgreSQL databases. 
 This extension adds support for connecting to PostgreSql databases using the #!connect postgres magic command.

## Working on TPCH SF10 ORDERS (15M rows)

### PGSQL 16 to PG 17

In [3]:
// Connect to PostgreSQL target database
#!connect postgres "Host=localhost;Port=25432;Database=tpch;Username=FastUser;Password=FastPassword" --kernel-name pgsql17

Kernel added: #!sql-pgsql17

#### Target orders_3 with a clustered ColumnStoreIndex

In [3]:
TRUNCATE TABLE tpch_10.orders;
SELECT count(*) count_target_table_rows from tpch_10.orders;

ALTER TABLE tpch_10.orders SET UNLOGGED;

count_target_table_rows
0


### Describe Oracle Source Table
```

In [24]:
echo 'DESC tpch_in.orders_flat;' | sqlplus -S TPCH_IN/TPCH_IN@//localhost:1521/ORCLPDB

 Nom					   NULL ?   Type
 ----------------------------------------- -------- ----------------------------
 O_ORDERKEY				   NOT NULL NUMBER(38)
 O_CUSTKEY				   NOT NULL NUMBER(19)
 O_ORDERSTATUS					    CHAR(1)
 O_TOTALPRICE					    NUMBER(19,3)
 O_ORDERDATE					    DATE
 O_ORDERPRIORITY				    CHAR(15)
 O_CLERK					    CHAR(15)
 O_SHIPPRIORITY 				    NUMBER(38)
 O_COMMENT					    VARCHAR2(79)



## Test with Method RowId (oracle source only)

In Oracle, a ROWID is a unique physical address identifying a row’s exact storage location.

You can SELECT it and even filter data using this hidden column. FastTransfer use the RowId column to select data by chunck on oracle source.

In [8]:

cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "oraodp" `
--sourceserver "localhost:1521/ORCLPDB" `
--sourcedatabase "ORCLPDB" `
--sourceuser "TPCH_IN" `
--sourcepassword "TPCH_IN" `
--sourceschema "TPCH_IN" `
--sourcetable "ORDERS_FLAT" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "orders" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Rowid" `
--degree -2 `
--runid "ORA21_to_PG17"

2025-09-22T11:21:36.749 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- FastTransfer - running in trial mode – will expires on 2025‑10‑20 (28 day(s) left).
2025-09-22T11:21:36.759 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Starting
2025-09-22T11:21:36.759 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- FastTransfer Version : 0.13.10.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-22T11:21:36.760 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- OS : Microsoft Windows 10.0.26100
2025-09-22T11:21:36.760 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Process ID : 39064
2025-09-22T11:21:36.760 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Provided Run ID : ORA21_to_PG17
2025-09-22T11:21:36.760 +02:00 -|- FastTransfer -|- ORA21_to_PG17 -|- INFORMATION -|

In [8]:
SELECT count(*) count_target_table_rows from tpch_10.orders;

count_target_table_rows
15000000


In [17]:
SELECT * FROM tpch_10.orders LIMIT 5;

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
24552903,1313752,F,305754.99,1992-03-05 00:00:00Z,3-MEDIUM,Clerk#000007605,0,ly regular requests within the furiously regul
24575648,978844,F,175154.42,1992-12-10 00:00:00Z,2-HIGH,Clerk#000001182,0,y across the requests. quickly special accounts cajole fluffily
24575651,624299,F,116155.58,1992-02-09 00:00:00Z,4-NOT SPECIFIED,Clerk#000009852,0,ironic packages. fluffily express courts are across the slyly ironic theodo
24575682,201691,F,116158.29,1992-06-14 00:00:00Z,4-NOT SPECIFIED,Clerk#000004253,0,t the blithely final pinto beans; furio
24575712,1435832,F,330511.13,1992-11-27 00:00:00Z,1-URGENT,Clerk#000008569,0,its. regular foxes boost quickly. slyly special dugouts sleep blithe


#### Target orders_heap : Heap Table

In [10]:
DROP TABLE IF EXISTS tpch_10.orders_heap;

CREATE TABLE tpch_10.orders_heap AS
SELECT * 
FROM tpch_10.orders
WHERE 1=0;

ALTER TABLE tpch_10.orders_heap SET UNLOGGED;

SELECT count(*) count_orders_heap from tpch_10.orders_heap;

count_orders_heap
0


In [11]:

cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "pgcopy" `
--sourceserver "localhost:15432" `
--sourcedatabase "tpch" `
--sourceuser "FastUser" `
--sourcepassword "FastPassword" `
--sourceschema "tpch_10" `
--sourcetable "orders" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "orders_heap" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ctid" `
--degree -2 `
--runid "PG16_to_PG17_heap"

2025-09-22T00:33:56.757 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- DEBUG -|- tpch.tpch_10.orders_heap -|- Using log settings from FastTransfer_Settings.json
2025-09-22T00:33:56.767 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- FastTransfer – running in normal mode.
2025-09-22T00:33:56.784 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- License for Architecture & Performance validated successfully. License valid until 2026-09-05
2025-09-22T00:33:56.785 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- Starting
2025-09-22T00:33:56.785 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- FastTransfer Version : 0.13.9.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-22T00:33:56.785 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- OS : Microsoft Windows

In [1]:

SELECT count(*) count_rows_in_heap from tpch_10.orders_heap;

count_rows_in_heap
15000000


In [16]:
SELECT * from tpch_10.orders_heap LIMIT 5;

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
33660512,775268,F,57043.01,1994-09-30 00:00:00Z,4-NOT SPECIFIED,Clerk#000003931,0,"special, regular requests use."
33660514,1403410,F,69873.94,1994-04-22 00:00:00Z,4-NOT SPECIFIED,Clerk#000008318,0,final deposits abou
41281026,1211515,F,180962.41,1994-09-01 00:00:00Z,4-NOT SPECIFIED,Clerk#000005558,0,lithely final packages cajole daring
41281028,340883,F,243024.32,1994-04-23 00:00:00Z,1-URGENT,Clerk#000000631,0,nding packages. carefully bold pinto beans are furiously busily
33660550,192962,F,35781.16,1994-01-19 00:00:00Z,2-HIGH,Clerk#000004242,0,"final accounts sleep final, ironic pin"


#### Test using a bigger table : LineItem (60M Rows x 16 columns - 16GB) 

In [18]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "pgcopy" `
--sourceserver "localhost:15432" `
--sourcedatabase "tpch" `
--sourceuser "FastUser" `
--sourcepassword "FastPassword" `
--sourceschema "tpch_10" `
--sourcetable "lineitem" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "lineitem_heap_unlogged" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ctid" `
--degree -2 `
--runid "PG16_to_PG17_60M_rowsx16_columns"

2025-09-22T00:45:50.942 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- DEBUG -|- tpch.tpch_10.lineitem_heap_unlogged -|- Using log settings from FastTransfer_Settings.json
2025-09-22T00:45:50.954 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- FastTransfer – running in normal mode.
2025-09-22T00:45:50.973 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- License for Architecture & Performance validated successfully. License valid until 2026-09-05
2025-09-22T00:45:50.973 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- Starting
2025-09-22T00:45:50.973 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- FastTransfer Version : 0.13.9.0 Architecture : X64 - Framework : .NET 8.0.19
2025-09-22T

#### Results with Ctid :

| PG16 ==> PG17 (Ctid method) 				| Elapsed Time (seconds)|Cells/sec 	|Parallel Dispatch Method
|-----------------------------------------------------------------------|:---------------------:|----------:| :--: |
| FastTransfer Load LINEITEM (60M Rows x 16c columns) HEAP Target       | 17.1     			|56_140_350    | Ctid |
| FastTransfer Load ORDERS   (15M Rows x  9c columns) HEAP Target 		| 3.7     				|36_486_486    | Ctid |


The speed (in cells/sec) is more important with LINEITEM load because the table is bigger and the startup time take a smaller role/space in the total time than it take with ORDERS load.